In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda, GRU
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/absolute_periods750_250_240.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
def normalize_data(df):
    """normalize a dataframe."""
    mean = df.mean(axis=1)
    std = df.std(axis=1)
    df = df.sub(mean, axis=0)
    df = df.div(std, axis=0)
    df = df.values
    return df
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [7]:
i = 0
timestep = 240

In [8]:
x_train = dataset[0][i][0]['AMZN'].values * 1000
y_train = dataset[0][i][1]['AMZN'].values * 1.0
x_test = dataset[1][i][0]['AMZN'].values
y_test = dataset[1][i][1]['AMZN'].values * 1.0

In [9]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {x_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (750,)
y train shape: (750,)
x test shape: (490,)
y test shape: (490,)


In [10]:
i = 0
x_train1 = normalize_data(dataset[0][i][0])
# y_train1 = get_one_hot(dataset[0][i][1].values, 2) * 1.0
y_train1 = dataset[0][i][1].values * 1.0
x_test1 = normalize_data(dataset[1][i][0])
y_test1 = get_one_hot(dataset[1][i][1].values, 2) * 1.0

In [11]:
print(f"x train 1 shape: {x_train1.shape}")
print(f"y train 1 shape: {y_train1.shape}")
print(f"x test 1 shape: {x_test1.shape}")
print(f"y test 1 shape: {y_test1.shape}")

x train 1 shape: (750, 31)
y train 1 shape: (750, 31)
x test 1 shape: (490, 31)
y test 1 shape: (490, 31, 2)


In [12]:
x_series = [x_train[i:i+240] for i in range(750 - 240)]
y_series = [y_train[i+240] for i in range(750 - 240)]
# x_series = [x_train1[i:i+240, 0] for i in range(750 - 240)]
# y_series = [y_train1[i+240, 0] for i in range(750 - 240)]
x = np.array(x_series)
y = np.array(y_series)
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (510, 240)
y shape: (510,)


In [13]:
# x = x.transpose((0,2,1))
# x = np.reshape(x, (510 * 31, timestep))
# y = np.reshape(y, (510 * 31, 2))
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (510, 240)
y shape: (510,)


In [14]:
x = np.reshape(x, (x.shape[0], x.shape[1], 1))
print(f"x shape: {x.shape}")


x shape: (510, 240, 1)


In [15]:

x_train = normalize_data(dataset[0][i][0])
y_train = get_one_hot(dataset[0][i][1].values, 2) * 1.0
x_test = normalize_data(dataset[1][i][0])
y_test = get_one_hot(dataset[1][i][1].values, 2) * 1.0

train_gen = TimeseriesGenerator(x_train, y_train,
                                length=timestep, sampling_rate=1,
                                batch_size=510)
test_gen = TimeseriesGenerator(x_test, y_test,
                               length=timestep, sampling_rate=1,
                               batch_size=250)

In [16]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {y_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31, 2)
y test shape: (490, 31, 2)


In [17]:
x_train = train_gen[0][0]
y_train = train_gen[0][1]
y_test = test_gen[0][0]
y_test = test_gen[0][1]

In [18]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {y_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 31, 2)
y test shape: (250, 31, 2)


In [19]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [20]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {y_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 31, 2)
y test shape: (250, 31, 2)


In [22]:
# expected input data shape: (batch_size, timesteps, data_dim)
regressor = Sequential()
# regressor.add(LSTM(units=25, input_shape=(timestep, 1), dropout=0.1))
# regressor.add(GRU(units=25, return_sequences=True, input_shape=(timestep, 1), dropout=0.3, recurrent_dropout=0.3))
# regressor.add(GRU(25, return_sequences=True, dropout=0.3))
# regressor.add(GRU(25, dropout=0.3, recurrent_dropout=0.3))

regressor.add(LSTM(units=25, input_shape=(timestep, 1), dropout=0.1, recurrent_dropout=0.1))
# regressor.add(LSTM(25, return_sequences=True, dropout=0.1))
# regressor.add(LSTM(25, return_sequences=True, dropout=0.1))
# regressor.add(LSTM(50, dropout=0.3, recurrent_dropout=0.3))

# regressor.add(Dense(10,input_shape=(timestep, ), activation='relu'))
# regressor.add(Dense(100, activation='relu'))
# regressor.add(Dense(100, activation='relu'))

# regressor.add(Reshape((31, 2)))
# regressor.add(Lambda(lambda x: softmax(x, axis=-1)))

# regressor.add(Dense(2, activation='softmax'))
regressor.add(Dense(1, activation='sigmoid'))
# regressor.compile(loss='mean_squared_error',
#                   optimizer='rmsprop',
#                   metrics=['accuracy'])
regressor.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
regressor.fit(x, y, epochs=1000,batch_size=1000, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=1000),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

# regressor.fit_generator(train_gen, steps_per_epoch=len(train_gen),
#                         epochs=10, validation_data=test_gen,
#                         callbacks=[
#                             EarlyStopping(monitor='val_loss',
#                                           mode='min', patience=10),
#                             ModelCheckpoint(filepath="../model/mymodel.h5",
#                                             monitor='val_acc',
#                                             save_best_only=True)])


Train on 408 samples, validate on 102 samples
Epoch 1/1000
408/408 [==============================] - 1s 2ms/step - loss: 0.7727 - acc: 0.4951 - val_loss: 0.8440 - val_acc: 0.4314
Epoch 2/1000
408/408 [==============================] - 0s 685us/step - loss: 0.7667 - acc: 0.4853 - val_loss: 0.8288 - val_acc: 0.4314
Epoch 3/1000
408/408 [==============================] - 0s 630us/step - loss: 0.7497 - acc: 0.4828 - val_loss: 0.8172 - val_acc: 0.4314
Epoch 4/1000
408/408 [==============================] - 0s 667us/step - loss: 0.7404 - acc: 0.5098 - val_loss: 0.8082 - val_acc: 0.4314
Epoch 5/1000
408/408 [==============================] - 0s 707us/step - loss: 0.7452 - acc: 0.4853 - val_loss: 0.8011 - val_acc: 0.4314
Epoch 6/1000
408/408 [==============================] - 0s 721us/step - loss: 0.7332 - acc: 0.4926 - val_loss: 0.7934 - val_acc: 0.4314
Epoch 7/1000
408/408 [==============================] - 0s 667us/step - loss: 0.7137 - acc: 0.5319 - val_loss: 0.7888 - val_acc: 0.4314
Epoc

Epoch 61/1000
408/408 [==============================] - 0s 736us/step - loss: 0.6755 - acc: 0.5809 - val_loss: 0.7003 - val_acc: 0.5000
Epoch 62/1000
408/408 [==============================] - 0s 790us/step - loss: 0.6724 - acc: 0.5784 - val_loss: 0.6996 - val_acc: 0.5098
Epoch 63/1000
408/408 [==============================] - 0s 808us/step - loss: 0.6745 - acc: 0.5760 - val_loss: 0.6989 - val_acc: 0.5098
Epoch 64/1000
408/408 [==============================] - 0s 731us/step - loss: 0.6721 - acc: 0.5956 - val_loss: 0.7005 - val_acc: 0.5098
Epoch 65/1000
408/408 [==============================] - 0s 730us/step - loss: 0.6718 - acc: 0.5907 - val_loss: 0.7002 - val_acc: 0.5196
Epoch 66/1000
408/408 [==============================] - 0s 716us/step - loss: 0.6727 - acc: 0.5735 - val_loss: 0.6990 - val_acc: 0.5196
Epoch 67/1000
408/408 [==============================] - 0s 766us/step - loss: 0.6745 - acc: 0.5686 - val_loss: 0.6970 - val_acc: 0.4902
Epoch 68/1000
408/408 [==================

Epoch 121/1000
408/408 [==============================] - 0s 771us/step - loss: 0.6633 - acc: 0.5735 - val_loss: 0.6875 - val_acc: 0.5196
Epoch 122/1000
408/408 [==============================] - 0s 742us/step - loss: 0.6579 - acc: 0.6054 - val_loss: 0.6850 - val_acc: 0.5294
Epoch 123/1000
408/408 [==============================] - 0s 710us/step - loss: 0.6524 - acc: 0.6225 - val_loss: 0.6858 - val_acc: 0.5294
Epoch 124/1000
408/408 [==============================] - 0s 721us/step - loss: 0.6595 - acc: 0.5858 - val_loss: 0.6847 - val_acc: 0.5196
Epoch 125/1000
408/408 [==============================] - 0s 739us/step - loss: 0.6639 - acc: 0.5931 - val_loss: 0.6835 - val_acc: 0.5392
Epoch 126/1000
408/408 [==============================] - 0s 701us/step - loss: 0.6559 - acc: 0.6127 - val_loss: 0.6849 - val_acc: 0.5196
Epoch 127/1000
408/408 [==============================] - 0s 666us/step - loss: 0.6529 - acc: 0.6225 - val_loss: 0.6823 - val_acc: 0.5490
Epoch 128/1000
408/408 [==========

Epoch 181/1000
408/408 [==============================] - 0s 653us/step - loss: 0.6462 - acc: 0.6078 - val_loss: 0.6838 - val_acc: 0.5588
Epoch 182/1000
408/408 [==============================] - 0s 647us/step - loss: 0.6509 - acc: 0.6176 - val_loss: 0.6832 - val_acc: 0.5882
Epoch 183/1000
408/408 [==============================] - 0s 664us/step - loss: 0.6405 - acc: 0.6324 - val_loss: 0.6836 - val_acc: 0.5784
Epoch 184/1000
408/408 [==============================] - 0s 668us/step - loss: 0.6437 - acc: 0.6152 - val_loss: 0.6852 - val_acc: 0.5784
Epoch 185/1000
408/408 [==============================] - 0s 652us/step - loss: 0.6513 - acc: 0.6103 - val_loss: 0.6858 - val_acc: 0.5686
Epoch 186/1000
408/408 [==============================] - 0s 642us/step - loss: 0.6439 - acc: 0.6471 - val_loss: 0.6866 - val_acc: 0.5588
Epoch 187/1000
408/408 [==============================] - 0s 668us/step - loss: 0.6358 - acc: 0.6544 - val_loss: 0.6845 - val_acc: 0.5686
Epoch 188/1000
408/408 [==========

Epoch 241/1000
408/408 [==============================] - 0s 721us/step - loss: 0.6290 - acc: 0.6348 - val_loss: 0.6854 - val_acc: 0.5392
Epoch 242/1000
408/408 [==============================] - 0s 677us/step - loss: 0.6400 - acc: 0.6250 - val_loss: 0.6848 - val_acc: 0.5294
Epoch 243/1000
408/408 [==============================] - 0s 668us/step - loss: 0.6426 - acc: 0.6176 - val_loss: 0.6858 - val_acc: 0.5392
Epoch 244/1000
408/408 [==============================] - 0s 613us/step - loss: 0.6318 - acc: 0.6275 - val_loss: 0.6875 - val_acc: 0.5196
Epoch 245/1000
408/408 [==============================] - 0s 691us/step - loss: 0.6352 - acc: 0.6201 - val_loss: 0.6868 - val_acc: 0.5294
Epoch 246/1000
408/408 [==============================] - 0s 851us/step - loss: 0.6302 - acc: 0.6127 - val_loss: 0.6878 - val_acc: 0.5392
Epoch 247/1000
408/408 [==============================] - 0s 772us/step - loss: 0.6414 - acc: 0.6054 - val_loss: 0.6888 - val_acc: 0.5294
Epoch 248/1000
408/408 [==========

...

Epoch 721/1000
408/408 [==============================] - 0s 713us/step - loss: 0.4959 - acc: 0.7549 - val_loss: 0.8328 - val_acc: 0.5392
Epoch 722/1000
408/408 [==============================] - 0s 681us/step - loss: 0.4827 - acc: 0.7843 - val_loss: 0.8372 - val_acc: 0.5392
Epoch 723/1000
408/408 [==============================] - 0s 646us/step - loss: 0.4911 - acc: 0.7574 - val_loss: 0.8338 - val_acc: 0.5392
Epoch 724/1000
408/408 [==============================] - 0s 677us/step - loss: 0.4809 - acc: 0.7794 - val_loss: 0.8348 - val_acc: 0.5392
Epoch 725/1000
408/408 [==============================] - 0s 667us/step - loss: 0.4938 - acc: 0.7549 - val_loss: 0.8297 - val_acc: 0.5294
Epoch 726/1000
408/408 [==============================] - 0s 661us/step - loss: 0.4883 - acc: 0.7623 - val_loss: 0.8263 - val_acc: 0.5196
Epoch 727/1000
408/408 [==============================] - 0s 707us/step - loss: 0.5094 - acc: 0.7402 - val_loss: 0.8407 - val_acc: 0.5294
Epoch 728/1000
408/408 [==========

Epoch 781/1000
408/408 [==============================] - 0s 691us/step - loss: 0.4987 - acc: 0.7598 - val_loss: 0.8572 - val_acc: 0.5490
Epoch 782/1000
408/408 [==============================] - 0s 669us/step - loss: 0.4707 - acc: 0.7745 - val_loss: 0.8454 - val_acc: 0.5392
Epoch 783/1000
408/408 [==============================] - 0s 685us/step - loss: 0.4634 - acc: 0.7745 - val_loss: 0.8664 - val_acc: 0.5392
Epoch 784/1000
408/408 [==============================] - 0s 642us/step - loss: 0.4826 - acc: 0.7475 - val_loss: 0.8477 - val_acc: 0.5490
Epoch 785/1000
408/408 [==============================] - 0s 655us/step - loss: 0.4824 - acc: 0.7794 - val_loss: 0.8597 - val_acc: 0.5392
Epoch 786/1000
408/408 [==============================] - 0s 670us/step - loss: 0.4602 - acc: 0.7574 - val_loss: 0.8764 - val_acc: 0.5490
Epoch 787/1000
408/408 [==============================] - 0s 653us/step - loss: 0.4796 - acc: 0.7745 - val_loss: 0.8533 - val_acc: 0.5588
Epoch 788/1000
408/408 [==========

Epoch 841/1000
408/408 [==============================] - 0s 672us/step - loss: 0.4775 - acc: 0.7696 - val_loss: 0.8586 - val_acc: 0.5490
Epoch 842/1000
408/408 [==============================] - 0s 667us/step - loss: 0.4619 - acc: 0.7892 - val_loss: 0.8779 - val_acc: 0.5588
Epoch 843/1000
408/408 [==============================] - 0s 660us/step - loss: 0.4769 - acc: 0.7696 - val_loss: 0.8706 - val_acc: 0.5686
Epoch 844/1000
408/408 [==============================] - 0s 631us/step - loss: 0.4742 - acc: 0.7917 - val_loss: 0.8693 - val_acc: 0.5588
Epoch 845/1000
408/408 [==============================] - 0s 655us/step - loss: 0.4955 - acc: 0.7426 - val_loss: 0.8776 - val_acc: 0.5588
Epoch 846/1000
408/408 [==============================] - 0s 665us/step - loss: 0.4739 - acc: 0.7794 - val_loss: 0.8863 - val_acc: 0.5588
Epoch 847/1000
408/408 [==============================] - 0s 656us/step - loss: 0.4555 - acc: 0.7843 - val_loss: 0.8831 - val_acc: 0.5588
Epoch 848/1000
408/408 [==========

Epoch 901/1000
408/408 [==============================] - 0s 652us/step - loss: 0.4504 - acc: 0.7721 - val_loss: 0.9095 - val_acc: 0.5980
Epoch 902/1000
408/408 [==============================] - 0s 670us/step - loss: 0.4616 - acc: 0.7525 - val_loss: 0.8743 - val_acc: 0.5588
Epoch 903/1000
408/408 [==============================] - 0s 654us/step - loss: 0.4573 - acc: 0.7843 - val_loss: 0.8835 - val_acc: 0.5686
Epoch 904/1000
408/408 [==============================] - 0s 661us/step - loss: 0.4662 - acc: 0.7623 - val_loss: 0.8657 - val_acc: 0.5588
Epoch 905/1000
408/408 [==============================] - 0s 646us/step - loss: 0.4561 - acc: 0.7868 - val_loss: 0.8884 - val_acc: 0.5784
Epoch 906/1000
408/408 [==============================] - 0s 709us/step - loss: 0.4497 - acc: 0.7917 - val_loss: 0.9107 - val_acc: 0.5882
Epoch 907/1000
408/408 [==============================] - 0s 836us/step - loss: 0.4480 - acc: 0.7843 - val_loss: 0.8946 - val_acc: 0.5784
Epoch 908/1000
408/408 [==========

Epoch 961/1000
408/408 [==============================] - 0s 654us/step - loss: 0.4327 - acc: 0.8064 - val_loss: 0.9102 - val_acc: 0.5686
Epoch 962/1000
408/408 [==============================] - 0s 639us/step - loss: 0.4391 - acc: 0.8015 - val_loss: 0.9096 - val_acc: 0.5686
Epoch 963/1000
408/408 [==============================] - 0s 684us/step - loss: 0.4593 - acc: 0.7745 - val_loss: 0.9182 - val_acc: 0.5784
Epoch 964/1000
408/408 [==============================] - 0s 680us/step - loss: 0.4282 - acc: 0.8186 - val_loss: 0.9113 - val_acc: 0.5784
Epoch 965/1000
408/408 [==============================] - 0s 653us/step - loss: 0.4372 - acc: 0.7794 - val_loss: 0.9202 - val_acc: 0.5784
Epoch 966/1000
408/408 [==============================] - 0s 641us/step - loss: 0.4459 - acc: 0.8088 - val_loss: 0.9018 - val_acc: 0.5686
Epoch 967/1000
408/408 [==============================] - 0s 638us/step - loss: 0.4428 - acc: 0.7892 - val_loss: 0.9098 - val_acc: 0.5784
Epoch 968/1000
408/408 [==========